In [5]:
import sqlite3
import pandas as pd
import numpy as np

In [6]:
conn = sqlite3.connect('aggregate.db')
c = conn.cursor()

In [7]:
#imported all the IMDB files first

#name of people on titles. known_for_titles matches the title_id
imdb_name = pd.read_csv('zippedData/imdb.name.basics.csv.gz') 

#title, release region, language
imdb_title_a =  pd.read_csv('zippedData/imdb.title.akas.csv.gz') 

#title, year, runtime, genre
imdb_title_b =  pd.read_csv('zippedData/imdb.title.basics.csv.gz') 

#tile, director and writer using the title and name ids
imdb_title_c = pd.read_csv('zippedData/imdb.title.crew.csv.gz') 

#categorizes the people(nameid) on the movie(titleid) and includes their job role and characters they played
imdb_title_p = pd.read_csv('zippedData/imdb.title.principals.csv.gz') 

#average rating and num votes per title
imdb_title_r = pd.read_csv('zippedData/imdb.title.ratings.csv.gz') 

In [28]:
#used to get the full release date for each movie movie
tmdb_movies = pd.read_csv('zippedData/tmdb.movies.csv.gz')

In [29]:
#used to get the movie gross data
bom = pd.read_csv('zippedData/bom.movie_gross.csv.gz') 


In [16]:
#check the column names

imdb_title_p.head()

,tconst,ordering,nconst,category,job,characters
0,tt0111414,1,nm0246005,actor,NaN,"[""The Man""]"
1,tt0111414,2,nm0398271,director,NaN,NaN
2,tt0111414,3,nm3739909,producer,producer,NaN
3,tt0323808,10,nm0059247,editor,NaN,NaN
4,tt0323808,1,nm3579312,actress,NaN,"[""Beth Boothby""]"


In [35]:
imdb_name.head()

,nconst,primary_name,birth_year,death_year,primary_profession,known_for_titles
0,nm0061671,Mary Ellen Bauder,NaN,NaN,"miscellaneous,production_manager,producer","tt0837562,tt2398241,tt0844471,tt0118553"
1,nm0061865,Joseph Bauer,NaN,NaN,"composer,music_department,sound_department","tt0896534,tt6791238,tt0287072,tt1682940"
2,nm0062070,Bruce Baum,NaN,NaN,"miscellaneous,actor,writer","tt1470654,tt0363631,tt0104030,tt0102898"
3,nm0062195,Axel Baumann,NaN,NaN,"camera_department,cinematographer,art_department","tt0114371,tt2004304,tt1618448,tt1224387"
4,nm0062798,Pete Baxter,NaN,NaN,"production_designer,art_department,set_decorator","tt0452644,tt0452692,tt3458030,tt2178256"


In [12]:
#check the column names

imdb_title_a.head()

,title_id,ordering,title,region,language,types,attributes,is_original_title
0,tt0369610,10,Джурасик свят,BG,bg,NaN,NaN,0.0
1,tt0369610,11,Jurashikku warudo,JP,NaN,imdbDisplay,NaN,0.0
2,tt0369610,12,Jurassic World: O Mundo dos Dinossauros,BR,NaN,imdbDisplay,NaN,0.0
3,tt0369610,13,O Mundo dos Dinossauros,BR,NaN,NaN,short title,0.0
4,tt0369610,14,Jurassic World,FR,NaN,imdbDisplay,NaN,0.0


In [13]:
imdb_title_b.head()

,tconst,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy"


In [14]:
#renamed the 'tconst' column to match the title_id column from imdb_title_a
imdb_title_b = imdb_title_b.rename(columns = {"tconst": "title_id"})

In [15]:
#join b and a
#joined these tables first to just get all of the title info and the region in one table

title_ba = imdb_title_b.merge(imdb_title_a, on = 'title_id', how = 'left')
title_ba.head()

,title_id,primary_title,original_title,start_year,runtime_minutes,genres,ordering,title,region,language,types,attributes,is_original_title
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",1.0,Sangharsh,IN,hi,NaN,alternative transliteration,0.0
1,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",2.0,Sunghursh,NaN,NaN,original,NaN,1.0
2,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",3.0,Sunghursh,IN,NaN,NaN,NaN,0.0
3,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",4.0,Sunghursh,IN,hi,NaN,alternative transliteration,0.0
4,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",5.0,Sungharsh,IN,hi,NaN,alternative spelling,0.0


In [17]:
#created a separate df that joins the imdb_name with imdb_title_p
#joined these tables to get the name and the specific profession ('category') they had on that particular title

name_princ = imdb_name.merge(imdb_title_p, on = 'nconst', how = 'left') #used 'nconst' to match the names

In [18]:
#renamed the 'tconst' column to match the title_id column and use that later to join with title_ba table
name_princ = name_princ.rename(columns = {"tconst": "title_id"}) 
 

In [19]:
name_princ.head()

,nconst,primary_name,birth_year,death_year,primary_profession,known_for_titles,title_id,ordering,category,job,characters
0,nm0061671,Mary Ellen Bauder,NaN,NaN,"miscellaneous,production_manager,producer","tt0837562,tt2398241,tt0844471,tt0118553",tt2398241,9.0,producer,producer,NaN
1,nm0061865,Joseph Bauer,NaN,NaN,"composer,music_department,sound_department","tt0896534,tt6791238,tt0287072,tt1682940",tt0433397,7.0,composer,NaN,NaN
2,nm0061865,Joseph Bauer,NaN,NaN,"composer,music_department,sound_department","tt0896534,tt6791238,tt0287072,tt1682940",tt1681372,8.0,composer,NaN,NaN
3,nm0061865,Joseph Bauer,NaN,NaN,"composer,music_department,sound_department","tt0896534,tt6791238,tt0287072,tt1682940",tt2387710,8.0,composer,NaN,NaN
4,nm0061865,Joseph Bauer,NaN,NaN,"composer,music_department,sound_department","tt0896534,tt6791238,tt0287072,tt1682940",tt2281215,7.0,composer,NaN,NaN


In [20]:
#join both tables to get the name and the title info together
name_titles = title_ba.merge(name_princ, on = 'title_id', how = 'left')
name_titles.head()

,title_id,primary_title,original_title,start_year,runtime_minutes,genres,ordering_x,title,region,language,...,nconst,primary_name,birth_year,death_year,primary_profession,known_for_titles,ordering_y,category,job,characters
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",1.0,Sangharsh,IN,hi,...,nm0347899,Gulzar,1936.0,NaN,"music_department,writer,soundtrack","tt0091256,tt0178186,tt1010048,tt2176013",8.0,writer,dialogue,NaN
1,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",1.0,Sangharsh,IN,hi,...,nm1194313,Mahasweta Devi,1926.0,2016.0,writer,"tt0108001,tt0832902,tt0063540,tt0178562",7.0,writer,story,NaN
2,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",1.0,Sangharsh,IN,hi,...,nm0712540,Harnam Singh Rawail,1921.0,2004.0,"director,writer,producer","tt0266757,tt0266712,tt0134996,tt0266765",5.0,director,NaN,NaN
3,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",1.0,Sangharsh,IN,hi,...,nm0904537,Vyjayanthimala,1933.0,NaN,"actress,music_department,miscellaneous","tt0047990,tt0054910,tt0050665,tt0058547",2.0,actress,NaN,"[""Munni"",""Laila-E-Aasmaan""]"
4,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",1.0,Sangharsh,IN,hi,...,nm0023551,Abrar Alvi,1927.0,2009.0,"writer,actor,director","tt0071811,tt0359496,tt0056436,tt0061046",6.0,writer,dialogue,NaN


In [21]:
#testing out the new df

#groups the df by title and by category columns. the get_group method specifically shows actors in Iron Man 2
name_titles.groupby(['title', 'category']).get_group(('Iron Man 2', 'actor'))

,title_id,primary_title,original_title,start_year,runtime_minutes,genres,ordering_x,title,region,language,...,nconst,primary_name,birth_year,death_year,primary_profession,known_for_titles,ordering_y,category,job,characters
155524,tt1228705,Iron Man 2,Iron Man 2,2010,124.0,"Action,Adventure,Sci-Fi",11.0,Iron Man 2,AR,NaN,...,nm0000620,Mickey Rourke,1952.0,NaN,"actor,writer,music_department","tt1228705,tt0092563,tt1125849,tt0401792",2.0,actor,NaN,"[""Ivan Vanko""]"
155525,tt1228705,Iron Man 2,Iron Man 2,2010,124.0,"Action,Adventure,Sci-Fi",11.0,Iron Man 2,AR,NaN,...,nm0000332,Don Cheadle,1964.0,NaN,"actor,producer,soundtrack","tt0395169,tt0988047,tt0375679,tt1540133",4.0,actor,NaN,"[""Lt. Col. James 'Rhodey' Rhodes""]"
155529,tt1228705,Iron Man 2,Iron Man 2,2010,124.0,"Action,Adventure,Sci-Fi",11.0,Iron Man 2,AR,NaN,...,nm0000375,Robert Downey Jr.,1965.0,NaN,"actor,producer,soundtrack","tt0848228,tt1300854,tt0988045,tt0371746",1.0,actor,NaN,"[""Tony Stark""]"
155564,tt1228705,Iron Man 2,Iron Man 2,2010,124.0,"Action,Adventure,Sci-Fi",15.0,Iron Man 2,FR,NaN,...,nm0000620,Mickey Rourke,1952.0,NaN,"actor,writer,music_department","tt1228705,tt0092563,tt1125849,tt0401792",2.0,actor,NaN,"[""Ivan Vanko""]"
155565,tt1228705,Iron Man 2,Iron Man 2,2010,124.0,"Action,Adventure,Sci-Fi",15.0,Iron Man 2,FR,NaN,...,nm0000332,Don Cheadle,1964.0,NaN,"actor,producer,soundtrack","tt0395169,tt0988047,tt0375679,tt1540133",4.0,actor,NaN,"[""Lt. Col. James 'Rhodey' Rhodes""]"
155569,tt1228705,Iron Man 2,Iron Man 2,2010,124.0,"Action,Adventure,Sci-Fi",15.0,Iron Man 2,FR,NaN,...,nm0000375,Robert Downey Jr.,1965.0,NaN,"actor,producer,soundtrack","tt0848228,tt1300854,tt0988045,tt0371746",1.0,actor,NaN,"[""Tony Stark""]"
155574,tt1228705,Iron Man 2,Iron Man 2,2010,124.0,"Action,Adventure,Sci-Fi",16.0,Iron Man 2,GR,NaN,...,nm0000620,Mickey Rourke,1952.0,NaN,"actor,writer,music_department","tt1228705,tt0092563,tt1125849,tt0401792",2.0,actor,NaN,"[""Ivan Vanko""]"
155575,tt1228705,Iron Man 2,Iron Man 2,2010,124.0,"Action,Adventure,Sci-Fi",16.0,Iron Man 2,GR,NaN,...,nm0000332,Don Cheadle,1964.0,NaN,"actor,producer,soundtrack","tt0395169,tt0988047,tt0375679,tt1540133",4.0,actor,NaN,"[""Lt. Col. James 'Rhodey' Rhodes""]"
155579,tt1228705,Iron Man 2,Iron Man 2,2010,124.0,"Action,Adventure,Sci-Fi",16.0,Iron Man 2,GR,NaN,...,nm0000375,Robert Downey Jr.,1965.0,NaN,"actor,producer,soundtrack","tt0848228,tt1300854,tt0988045,tt0371746",1.0,actor,NaN,"[""Tony Stark""]"
155584,tt1228705,Iron Man 2,Iron Man 2,2010,124.0,"Action,Adventure,Sci-Fi",17.0,Iron Man 2,IT,NaN,...,nm0000620,Mickey Rourke,1952.0,NaN,"actor,writer,music_department","tt1228705,tt0092563,tt1125849,tt0401792",2.0,actor,NaN,"[""Ivan Vanko""]"


In [22]:
#made a new df for specifically US movie info

#grouped by region and used get group to only show the US movies
us_name_titles = name_titles.groupby('region').get_group('US')

In [23]:
#tested it out to make sure everything still works
us_name_titles.groupby(['title', 'category']).get_group(('Iron Man 2', 'actor'))

,title_id,primary_title,original_title,start_year,runtime_minutes,genres,ordering_x,title,region,language,...,nconst,primary_name,birth_year,death_year,primary_profession,known_for_titles,ordering_y,category,job,characters
155724,tt1228705,Iron Man 2,Iron Man 2,2010,124.0,"Action,Adventure,Sci-Fi",2.0,Iron Man 2,US,NaN,...,nm0000620,Mickey Rourke,1952.0,NaN,"actor,writer,music_department","tt1228705,tt0092563,tt1125849,tt0401792",2.0,actor,NaN,"[""Ivan Vanko""]"
155725,tt1228705,Iron Man 2,Iron Man 2,2010,124.0,"Action,Adventure,Sci-Fi",2.0,Iron Man 2,US,NaN,...,nm0000332,Don Cheadle,1964.0,NaN,"actor,producer,soundtrack","tt0395169,tt0988047,tt0375679,tt1540133",4.0,actor,NaN,"[""Lt. Col. James 'Rhodey' Rhodes""]"
155729,tt1228705,Iron Man 2,Iron Man 2,2010,124.0,"Action,Adventure,Sci-Fi",2.0,Iron Man 2,US,NaN,...,nm0000375,Robert Downey Jr.,1965.0,NaN,"actor,producer,soundtrack","tt0848228,tt1300854,tt0988045,tt0371746",1.0,actor,NaN,"[""Tony Stark""]"


In [24]:
#test to see if we can get movies from a specific year
us_name_titles.groupby('start_year').get_group(2016)

,title_id,primary_title,original_title,start_year,runtime_minutes,genres,ordering_x,title,region,language,...,nconst,primary_name,birth_year,death_year,primary_profession,known_for_titles,ordering_y,category,job,characters
17156,tt0475290,"Hail, Caesar!","Hail, Caesar!",2016,106.0,"Comedy,Drama,Music",11.0,"Hail, Caesar! A Tale of The Christ",US,NaN,...,nm0001054,Joel Coen,1954.0,NaN,"producer,writer,director","tt2042568,tt0116282,tt6412452,tt1019452",6.0,director,NaN,NaN
17157,tt0475290,"Hail, Caesar!","Hail, Caesar!",2016,106.0,"Comedy,Drama,Music",11.0,"Hail, Caesar! A Tale of The Christ",US,NaN,...,nm0001980,Carter Burwell,1954.0,NaN,"composer,music_department,soundtrack","tt1403865,tt2402927,tt0116282,tt5027774",9.0,composer,NaN,NaN
17158,tt0475290,"Hail, Caesar!","Hail, Caesar!",2016,106.0,"Comedy,Drama,Music",11.0,"Hail, Caesar! A Tale of The Christ",US,NaN,...,nm0000982,Josh Brolin,1968.0,NaN,"actor,producer,writer","tt4154756,tt1175491,tt0477348,tt1791528",1.0,actor,NaN,"[""Eddie Mannix""]"
17159,tt0475290,"Hail, Caesar!","Hail, Caesar!",2016,106.0,"Comedy,Drama,Music",11.0,"Hail, Caesar! A Tale of The Christ",US,NaN,...,nm0001053,Ethan Coen,1957.0,NaN,"producer,writer,director","tt6412452,tt0116282,tt1019452,tt2042568",5.0,director,NaN,NaN
17160,tt0475290,"Hail, Caesar!","Hail, Caesar!",2016,106.0,"Comedy,Drama,Music",11.0,"Hail, Caesar! A Tale of The Christ",US,NaN,...,nm0271479,Eric Fellner,1959.0,NaN,"producer,actor,executive","tt1707386,tt0783233,tt0475276,tt2980516",8.0,producer,producer,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2918690,tt9069126,Supernatural Fandom,Supernatural Fandom,2016,NaN,Documentary,1.0,Supernatural Fandom The Movie,US,NaN,...,nm4267050,Ian Macleod,NaN,NaN,"editor,cinematographer,producer","tt1817293,tt3592568,tt3515662,tt2244620",8.0,editor,NaN,NaN
2942683,tt9387300,Girls Love,Girls Love,2016,61.0,Romance,2.0,Wrong Sex,US,NaN,...,nm10305177,He Jia Ying,NaN,NaN,actor,tt9387300,2.0,actor,NaN,"[""Xiao Rou""]"
2942684,tt9387300,Girls Love,Girls Love,2016,61.0,Romance,2.0,Wrong Sex,US,NaN,...,nm10305176,Miller,NaN,NaN,actor,NaN,1.0,actor,NaN,"[""Mi Le""]"
2942687,tt9387300,Girls Love,Girls Love,2016,61.0,Romance,4.0,Good Love,US,NaN,...,nm10305177,He Jia Ying,NaN,NaN,actor,tt9387300,2.0,actor,NaN,"[""Xiao Rou""]"


In [30]:
#created a new df that joins with tmdb_movies

#did this to also get the month and day for the release date. Based on the matching 'title'

us_ntd = us_name_titles.merge(tmdb_movies, on = 'title', how = 'left')

In [31]:
#test to make sure it still works

us_ntd.groupby(['title', 'category']).get_group(('Iron Man 2', 'actor'))

,title_id,primary_title,original_title_x,start_year,runtime_minutes,genres,ordering_x,title,region,language,...,characters,Unnamed: 0,genre_ids,id,original_language,original_title_y,popularity,release_date,vote_average,vote_count
13869,tt1228705,Iron Man 2,Iron Man 2,2010,124.0,"Action,Adventure,Sci-Fi",2.0,Iron Man 2,US,NaN,...,"[""Ivan Vanko""]",2.0,"[12, 28, 878]",10138.0,en,Iron Man 2,28.515,2010-05-07,6.8,12368.0
13870,tt1228705,Iron Man 2,Iron Man 2,2010,124.0,"Action,Adventure,Sci-Fi",2.0,Iron Man 2,US,NaN,...,"[""Lt. Col. James 'Rhodey' Rhodes""]",2.0,"[12, 28, 878]",10138.0,en,Iron Man 2,28.515,2010-05-07,6.8,12368.0
13874,tt1228705,Iron Man 2,Iron Man 2,2010,124.0,"Action,Adventure,Sci-Fi",2.0,Iron Man 2,US,NaN,...,"[""Tony Stark""]",2.0,"[12, 28, 878]",10138.0,en,Iron Man 2,28.515,2010-05-07,6.8,12368.0


In [32]:
#created a new df that joins with bom. Now we have the name, title, date, and gross info for all US movies

#did this to get the gross data. Based on the matching 'title'

us_ntdg = us_ntd.merge(bom, on = 'title', how = 'left')

In [33]:
#Huzzah it still works!
us_ntdg.groupby(['title', 'category']).get_group(('Iron Man 2', 'actor'))

,title_id,primary_title,original_title_x,start_year,runtime_minutes,genres,ordering_x,title,region,language,...,original_language,original_title_y,popularity,release_date,vote_average,vote_count,studio,domestic_gross,foreign_gross,year
13869,tt1228705,Iron Man 2,Iron Man 2,2010,124.0,"Action,Adventure,Sci-Fi",2.0,Iron Man 2,US,NaN,...,en,Iron Man 2,28.515,2010-05-07,6.8,12368.0,Par.,312400000.0,311500000,2010.0
13870,tt1228705,Iron Man 2,Iron Man 2,2010,124.0,"Action,Adventure,Sci-Fi",2.0,Iron Man 2,US,NaN,...,en,Iron Man 2,28.515,2010-05-07,6.8,12368.0,Par.,312400000.0,311500000,2010.0
13874,tt1228705,Iron Man 2,Iron Man 2,2010,124.0,"Action,Adventure,Sci-Fi",2.0,Iron Man 2,US,NaN,...,en,Iron Man 2,28.515,2010-05-07,6.8,12368.0,Par.,312400000.0,311500000,2010.0


In [34]:
#convert release date column from string to a datetime so we can use to manipulate the data 
us_ntdg['release_date'] = pd.to_datetime(us_ntdg['release_date'])

In [36]:
us_ntdg.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 408580 entries, 0 to 408579
Data columns (total 36 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   title_id            408580 non-null  object        
 1   primary_title       408580 non-null  object        
 2   original_title_x    408579 non-null  object        
 3   start_year          408580 non-null  int64         
 4   runtime_minutes     349472 non-null  float64       
 5   genres              405366 non-null  object        
 6   ordering_x          408580 non-null  float64       
 7   title               408580 non-null  object        
 8   region              408580 non-null  object        
 9   language            5440 non-null    object        
 10  types               66241 non-null   object        
 11  attributes          22973 non-null   object        
 12  is_original_title   408575 non-null  float64       
 13  nconst              408282 no

In [102]:
us_ntdg.to_csv('full_us_data.csv', index=False)